
### PPP Calculation of Csh, Total Sandstone Porosity, Sw in Payzone Using Laminated Aquifer Slope Method


Use this workflow having already computed CshAquifer(array),Rw,a,m,n from the picket plot in excel from the aquifer. Prepare arrays for:
- Gamma Ray values every 2ft of pay
- Nuetron and Density values every 2ft of pay
- Rt values every 2 ft of pay (no need to calculate Rss)

use https://apps.automeris.io/wpd/



In [16]:
import numpy as np
import pandas as pd


#- Gamma Shale: 1 value (local max) **NEAR THE PAY ZONE**
#- Gamma Sand: 1 value (local min) **APPROXIMATING AQUIFER**
#- Gamma Aquifer: 1 value (local min)
#- Gamma Pay: Gamma ray of the Pay Zone **[Array]**
#- Csh Aquifer: shale concentration of Aquifer 1 value **MUST BE AT SAME DEPTH AS Gamma Aquifer**

#returns [Array]: CshPay: shale concentration along the payzone

def Csh(GammaShale,GammaAquifer,GammaPay,CshAquifer):
    slope = (1-CshAquifer)/(GammaShale-GammaAquifer) #1 value for each well
    CshPay = slope*(GammaPay-GammaShale)+1 #distribution for each 2ft of pay
    return CshPay

def CshNew(GammaShale,GammaSand,GammaPay):
    CshPay = (GammaPay - GammaSand)/(GammaShale - GammaSand) #distribution for each 2ft of pay
    return CshPay

#-----------------------------------------------------------------------------------------
#NPay: nuetron porosity every 2 ft of pay [Array]
#DPay: density every 2 ft of pay [Array]
#NShale: nuetron porosity of pure shale (1 value)
#DShale: density porosity of pure shale (1 value)
#CshPay: input the CshPay [Array] from above

#returns [Array]: PhiTotal: corrected, total porosity in the pay zone

def TotalPorosity(NPay,DPay,NShale,DShale,CshPay):
    PhiDcorrected = (DPay-(CshPay*DShale))/(1-CshPay)
    PhiNcorrected = (NPay-(CshPay*NShale))/(1-CshPay)
    PhiTotal = ( ((PhiNcorrected**2)+(PhiDcorrected**2))/2 )**.5
    return PhiTotal
#-----------------------------------------------------------------------------------------

#Rw: single value from picket plot
#Rt: DEEP resistivity every 2 ft of pay [Array]
#phiTotal: input array from above
#a,m,n single values from picket plot:

#returns [2-D Array]: of Sw and Shc

def Saturations(Rw,Rt,phiTotal,a,m,n):
    Sw = ((Rw/Rt)*(a/(phiTotal**m)))**(1/n)
    Shc = 1-Sw
    return np.array([Sw,Shc])
#-----------------------------------------------------------------------------------------

#For loops using arrays: GammaPay,Rw,Npay,Dpay,Rt values. Return distribution, weighted 

data = pd.read_csv('template.csv')
GammaPay = np.array(data['GammaPay'])
Rw = np.array(data['Rw'])
Rt = np.array(data['Rt'])
NPay = np.array(data['Npay'])
DPay = np.array(data['Dpay'])


#GammaAquifer is assumed with the I-1 log, so if your pay zone is a Haliburton log, subtract 15 from API reading
GammaShale = 105
GammaAquifer = 60
GammaSand = 45
CshAquifer = 0.1667
NShale = .36 #.402
DShale = .18 #.23
a = 1
m = 2 
n = 2.5


CshPay = np.array([])
CshPayNew = np.array([])
for i in range(len(GammaPay)):
    calc = Csh(GammaShale,GammaAquifer,GammaPay[i],CshAquifer)
    CshPay = np.append(CshPay,calc)
    
    calcNew = CshNew(GammaShale,GammaSand,GammaPay[i])
    CshPayNew = np.append(CshPayNew,calcNew)

PorosityPay = np.array([])
PorosityPayNew = np.array([])
for i in range(len(CshPay)):
    calc1 = TotalPorosity(NPay[i],DPay[i],NShale,DShale,CshPay[i])
    PorosityPay = np.append(PorosityPay,calc1)
    
    calc1New = TotalPorosity(NPay[i],DPay[i],NShale,DShale,CshPayNew[i])
    PorosityPayNew = np.append(PorosityPayNew,calc1New)
    
WaterSaturationPay = np.array([])
WaterSaturationPayNew = np.array([])
OilSaturationPay = np.array([])
for i in range(len(PorosityPay)):
    calc2 = Saturations(Rw[i],Rt[i],PorosityPay[i],a,m,n)
    WaterSaturationPay = np.append(WaterSaturationPay,calc2[0])
    OilSaturationPay = np.append(OilSaturationPay,calc2[1])
    
    calc2New =  Saturations(Rw[i],Rt[i],PorosityPayNew[i],a,m,n)
    WaterSaturationPayNew = np.append(WaterSaturationPayNew,calc2New[0])
    
# print(PorosityPay)
# print(WaterSaturationPay)

data['PorosityPay'] = PorosityPay
data['WaterSaturation'] = WaterSaturationPay

data['PorosityPayNew'] = PorosityPayNew
data['WaterSaturationPayNew'] = WaterSaturationPayNew
# print(PorosityPayNew)
# print(WaterSaturationPayNew)

data.to_csv('/Users/AdrianSalinas/Desktop/resources/Senior design/SandExport.csv')

data




,Depth,GammaPay,Rw,Rt,Npay,Dpay,Sidewall Porosity,Sidewall Saturation,PorosityPayNew,WaterSaturationPayNew
0,7448,83.67,0.05,5.02,0.28,0.22,0.25,0.60,0.227796,0.516737
1,7450,72.45,0.05,5.17,0.29,0.27,0.25,0.55,0.294102,0.416288
2,7452,82.65,0.05,3.76,0.30,0.25,0.26,0.60,0.295750,0.470731
